In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle
from collections import defaultdict

from torch.utils.data import Dataset, DataLoader
from Mmetrics import *

import LTR
import datautil
import permutationgraph
import DTR
import EEL
import PPG
import PL

ds2020, _ = datautil.load_data(2020, verbose=False)
ds2019, _ = datautil.load_data(2019, verbose=False)

In [4]:
ltrmodel = LTR.MSE_model(layers=[ds2020.trfm.shape[1], 256, 256, 1], lr=0.001, optimizer=torch.optim.Adam, dropout=0.1)
ltrmodel.fit(ds2020, epochs=10, batch_size=100, verbose=False)
y_pred2020 = ltrmodel.predict(ds2020.tefm, ds2020.tedlr)
print('LTR performance ndcg@10 for 2020:', LTRMetrics(ds2020.telv,np.diff(ds2020.tedlr),y_pred2020).NDCG(10))

ltrmodel = LTR.MSE_model(layers=[ds2019.trfm.shape[1], 256, 256, 1], lr=0.001, optimizer=torch.optim.Adam, dropout=0.1)
ltrmodel.fit(ds2019, epochs=10, batch_size=100, verbose=False)
y_pred2019 = ltrmodel.predict(ds2019.tefm, ds2019.tedlr)
print('LTR performance ndcg@10 for 2019:', LTRMetrics(ds2019.telv,np.diff(ds2019.tedlr),y_pred2019).NDCG(10))

LTR performance ndcg@10 for 2020: 0.39552854999428505
LTR performance ndcg@10 for 2019: 0.8077619089872989


In [5]:
ds = ds2019
ypred = y_pred2019
df = pd.DataFrame({'lv':ds.telv, 'g':ds.teg, 'qid':np.repeat(ds.tedlr[:-1], np.diff(ds.tedlr)), 'y_pred':ypred})
with open('LTR2019.pkl', 'wb') as f:
    pickle.dump(df, f)
ds = ds2019
ypred = y_pred2019
df = pd.DataFrame({'lv':ds.telv, 'g':ds.teg, 'qid':np.repeat(ds.tedlr[:-1], np.diff(ds.tedlr)), 'y_pred':ypred})
with open('LTR2020.pkl', 'wb') as f:
    pickle.dump(df, f)

PicklingError: Can't pickle <class 'datautil.ltr'>: attribute lookup ltr on datautil failed

In [11]:

def learn_one_PPG(qid, verbose, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt):
    s, e = dlr[qid:qid+2]
    y_pred_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=y_pred[s:e], g=g[s:e], sorted_docs=y_pred[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = y_pred_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = grade_levels)
    learner = PPG.Learner(  PPG_mat=None, samples_cnt=samples_cnt, 
                                objective_ins=objective_ins, 
                                sorted_docs = sorted_docs_s, 
                                dlr = dlr_s,
#                                 intra = np.arange(g_s.shape[0]),
                                intra = g_s,
                                inter = np.repeat(dlr_s[:-1], np.diff(dlr_s)))
    vals = learner.fit(epochs, lr, verbose=verbose)
    return vals

def learn_all_PPG(y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt):
    sorted_docs = []
    
    for qid in trange(dlr.shape[0] - 1, leave=False):
#     for qid in range(dlr.shape[0] - 1):
        min_b = learn_one_PPG(qid, 0, y_pred, g, dlr, epochs, lr, exposure, grade_levels, samples_cnt, sessions_cnt)
        sorted_docs.append(min_b)
        

    # print(ndcg_dtr(exposure, lv, np.concatenate(y_rerank), dlr, g, query_counts))
    return sorted_docs



In [7]:
import time
epochs = 50

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])

for qid in range(8):
        start = time.time()
        learn_one_PPG(qid,0, y_pred2020, ds2020.teg, ds2020.tedlr, epochs, 0.1, exposure=exposure2020,
                grade_levels=5, samples_cnt=32, sessions_cnt=20)
        print(qid, 'len:', ds2020.tedlr[qid+1]-ds2020.tedlr[qid], 'took:', time.time()-start)

0 len: 11 took: 7.0557332038879395
1 len: 10 took: 3.656348705291748
2 len: 19 took: 13.886746883392334
3 len: 40 took: 27.484052658081055
4 len: 30 took: 68.45644283294678
5 len: 10 took: 6.936711072921753
6 len: 14 took: 6.6985132694244385
7 len: 20 took: 7.692243576049805


In [9]:

for qid in range(8):
        start = time.time()
        learn_one_PPG(qid,0, y_pred2020, ds2020.teg, ds2020.tedlr, epochs, 0.1, exposure=exposure2020,
                grade_levels=5, samples_cnt=32, sessions_cnt=20)
        print(qid, 'len:', ds2020.tedlr[qid+1]-ds2020.tedlr[qid], 'took:', time.time()-start)

0 len: 11 took: 4.1907336711883545
1 len: 10 took: 4.263021945953369
2 len: 19 took: 18.854395866394043
3 len: 40 took: 26.65814781188965
4 len: 30 took: 43.14603543281555
5 len: 10 took: 2.7891130447387695
6 len: 14 took: 3.9058539867401123
7 len: 20 took: 7.33855938911438


In [49]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
res = learn_all_PPG(y_pred2020, ds2020.teg, ds2020.tedlr, epochs, 0.1, exposure=exposure2020,
        grade_levels=5, samples_cnt=32, sessions_cnt=20)

KeyboardInterrupt: 

In [54]:
def evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    
    if metric == 'EEL':
        objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    else:
        objective_ins = DTR.DTR(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure)
        
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    ndcg = 0
    for i in range(sessions_cnt):
        ndcg += ((2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum() / idcg
        
    return objective_ins.eval(permutation), ndcg / sessions_cnt
 
def evaluate_all(metric, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res, ndcgs = [], [], [], []
    for qid in range(dlr.shape[0] - 1):
        s,e = ds.tedlr[qid:qid+2]
        if len(np.unique(g[s:e])) == 1:
            continue
        out1, ndcg = evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
#         eel, eer, eed = out1
        eel = out1
        eel_res.append(eel)
#         eer_res.append(eer)
#         eed_res.append(eed)
        ndcgs.append(ndcg)
    return np.array(eel_res), np.array(ndcgs)
#     return np.array(eel_res), np.array(eer_res), np.array(eed_res), np.array(ndcgs)

def estimated_evaluate_one(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 5)
    return objective_ins.eval_detailed(permutation)
 
def estimated_evaluate_all(lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res = [], [], []
    for qid in range(dlr.shape[0] - 1):
        s,e = ds.tedlr[qid:qid+2]
        if len(np.unique(g[s:e])) == 1:
            continue
        eel, eer, eed = estimated_evaluate_one(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
        eel_res.append(eel)
        eer_res.append(eer)
        eed_res.append(eed)
    return np.array(eel_res), np.array(eer_res), np.array(eed_res)

In [50]:
import pickle
import os

files = os.listdir('/ivi/ilps/personal/avardas/_data/PPG')
res = {}
# for alg in ['PPG', 'nointra_PPG', 'PL']:
#     with open(f'{alg}_EEL_results.pkl', 'rb') as f:
#         res[alg] = pickle.load(f)
for file in files:
    with open(f'/ivi/ilps/personal/avardas/_data/PPG/{file}', 'rb') as f:
        res[file[:-12]] = pickle.load(f)
    

In [55]:

exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.tedlr).max()+2)])
exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.tedlr).max()+2)])


for alg in res:
    _res = res[alg]
    print(alg, '-'*30)
    sessions_cnt = int(alg.split('_')[3])
    metric = alg.split('_')[4]
    for key in _res:
        if '2019' in key:
            ds = ds2019
            ypred = y_pred2019
            exposure = exposure2019
        else:
            ds = ds2020
            ypred = y_pred2020
            exposure = exposure2020
            
#         eel_res, eer_res, eed_res, ndcg = evaluate_all(ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=sessions_cnt)
        eel_res, ndcg = evaluate_all(metric, ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=sessions_cnt)
#         es_eel_res, es_eer_res, es_eed_res = estimated_evaluate_all(ypred, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=20)
    
        print(f'{key}', eel_res.mean(), '(', ndcg.mean(), ')')
#         print(f'{key}', eel_res.mean(), '(', es_eel_res.mean(), ')')
#         print(f'\t', ndcg.mean())
#         print(f'\t', eer_res.mean(), '(', es_eer_res.mean(), ')')
#         print(f'\t', eed_res.mean(), '(', es_eed_res.mean(), ')')


nointra_PL_16_2_EEL ------------------------------
2020_0.01 0.11060704034191648 ( 0.36412550893331896 )
2019_0.01 0.041132737707442774 ( 0.7624985154560728 )
2020_0.1 0.2274005108405296 ( 0.34387981689465474 )
2019_0.1 0.06794155460408947 ( 0.7634732795663499 )
nointra_PPG_16_16_DTR ------------------------------
2020_0.01 1.1254148155803896 ( 0.3831953567942967 )
2019_0.01 0.5561991832565913 ( 0.763395992621248 )
nointra_PL_8_1_EEL ------------------------------
2020_0.01 0.18408167122104327 ( 0.34751903381819693 )
2019_0.01 0.05202632730297503 ( 0.7579379422321025 )
2020_0.1 0.3122214467252436 ( 0.3612134184007923 )
2019_0.1 0.07587826534204901 ( 0.7658045153514828 )
2020_0.5 0.24919210285950813 ( 0.37300554854626383 )
2019_0.5 0.05732811905495799 ( 0.7601485243859107 )
intra_PPG_16_1_DTR ------------------------------
2020_0.01 1.1231033100444354 ( 0.36468786981270457 )
2019_0.01 0.5623747401264753 ( 0.8031187489987092 )
2020_0.1 1.1236297681676775 ( 0.3693359825459978 )
2019_0.1 0

2019_0.5 0.5603182691160047 ( 0.7571994742753481 )
intra_PPG_8_8_DTR ------------------------------
2020_0.01 1.1131742031395802 ( 0.39427271212876547 )
2019_0.01 0.5583848481012219 ( 0.7888082651145408 )
2020_0.1 1.116279459771534 ( 0.3957150106116559 )
2019_0.1 0.5591214845849252 ( 0.7905972659333833 )
2020_0.5 1.1234488124842124 ( 0.39187688048715025 )
2019_0.5 0.5571698204096374 ( 0.7902491131976084 )
nointra_PL_8_16_EEL ------------------------------
2020_0.01 0.10772610202272284 ( 0.3530368681973402 )
2019_0.01 0.024918622971092583 ( 0.761928531890181 )
2020_0.1 0.2033267546726065 ( 0.35299989765260664 )
2019_0.1 0.054211713079415025 ( 0.7658192884128038 )
2020_0.5 0.19440759896429652 ( 0.35035359737093413 )
2019_0.5 0.04832526477547355 ( 0.772345128630696 )
nointra_PL_16_1_EEL ------------------------------
2020_0.01 0.21189117151877407 ( 0.3493133935510833 )
2019_0.01 0.05989231407294966 ( 0.766380083263895 )
2020_0.1 0.23145985484360837 ( 0.3634685145295664 )
2019_0.1 0.081504

2019_0.01 0.04525282287238619 ( 0.7905165456753062 )
2020_0.1 0.18581367140778687 ( 0.38321102377841243 )
2019_0.1 0.04534701676695567 ( 0.790430286290082 )
2020_0.5 0.18575911997538366 ( 0.3938375181015517 )
2019_0.5 0.045262058756727785 ( 0.7809389624507364 )
nointra_PL_8_8_DTR ------------------------------
2020_0.01 1.1431686505576086 ( 0.3587966976815917 )
2019_0.01 0.5647313123709207 ( 0.7574270769487299 )
2020_0.1 1.1099734968577384 ( 0.35030357766870107 )
2019_0.1 0.5602928726198231 ( 0.7611615256162639 )
2020_0.5 1.1259852869256148 ( 0.3733533204088649 )
2019_0.5 0.5672906943757677 ( 0.7535605092953518 )
intra_PPG_16_8_EEL ------------------------------
2020_0.01 0.158632829711133 ( 0.39615006935012953 )
2019_0.01 0.04287086560446421 ( 0.8018507706595827 )
2020_0.1 0.158529962887589 ( 0.3962332924292197 )
2019_0.1 0.04298185880515514 ( 0.8014075659329946 )
2020_0.5 0.15821552344430273 ( 0.3932938101181713 )
2019_0.5 0.042722668521801926 ( 0.798267671896727 )
nointra_PPG_8_8_EE

In [40]:
def test(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    print(argsort)
    print(lv[s:e])
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    print('idcg:', idcg)
    ndcg = 0
    for i in range(sessions_cnt):
        dcg = ((2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
        print('dcg:', dcg)
        ndcg += dcg / idcg
        
    return objective_ins.eval_detailed(permutation), ndcg / sessions_cnt

In [52]:
# ds = ds2019
# exposure = exposure2020
# _res = res['PPG']
# key = '2019_0.01'
test(1,ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=20)

[9 3 8 7 6 5 4 2 1 0]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
idcg: 1.6309297535714575
dcg: 1.4306765580733931
dcg: 0.9306765580733931


ValueError: operands could not be broadcast together with shapes (0,) (10,) 

In [43]:
eel_res, eer_res, eed_res, ndcg = evaluate_all(ds.telv, ds.teg, ds.tedlr, _res[key], exposure, sessions_cnt=20)

In [44]:
ndcg

array(0.99428342)